# Overview 
 - I want to understand rental price behaviour in dublin against various features such as location and the size of the property.
 - I want to create a model to predict a rental price based on such attributes so that I could make a more informed decision, when I try to enter the rental market next year.
 

## Data Collection

[Rent.ie](https://www.rent.ie/) is Ireland's newest and freshest lettings website offering renters an alternative to the Daft lettings section when looking for rental accommodation and flatshares. They  are focused on 18-25 year olds and aim to be the top renting website for this demographic.

In the following cells, I created a scraper tool to scrape over 2,500 property descriptions using python and selenium.


In [38]:
rent_ie_url ="https://www.rent.ie/houses-to-let/renting_dublin/page_{0}/"
import geopy.distance
from copy import deepcopy
import googlemaps
import googleplaces
from pprint import pprint
import requests
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.remote.webelement import WebElement
import selenium.webdriver.support.ui as ui
import selenium.webdriver.support.expected_conditions as EC
import time
import os
import pandas as pd
import random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [39]:
options = webdriver.ChromeOptions()
options.add_argument("incognito")


driver = webdriver.Chrome(r'C:\Users\tonyr\chromedriver\chromedriver.exe', options=options)
time.sleep(2)


list_address = []
list_price = []
list_bathroom = []
list_bedroom = []
list_furnished = []
list_property_desc = []
list_week = []
list_month = []


In [43]:
for i in range(1,148):
    rent_url = rent_ie_url.format(i)
    driver.get(rent_url)
    time.sleep(4 + random.uniform(0,3))
    soup = BeautifulSoup(driver.page_source, "lxml")
    for adver in soup.find_all("div",class_="search_result"):
        address = adver.find("div", class_="sresult_address")

        list_address.append(address.find("a").get_text().strip())

        description = adver.find("div", class_="sresult_description")
        price_desc = description.find("h4").get_text().strip()
        if "month"  in price_desc.lower():
            list_month.append(1)
        else:
            list_month.append(0)
        if "week"  in  price_desc.lower():
            list_week.append(1)
        else:
            list_week.append(0)
        price = ""
        for item in price_desc:
            if item.isdigit():
                price += item
        list_price.append(int(price))
        
        list_property_desc.append((str(description.find("br").next).strip()))

        

        property_desc = description.find("h3").get_text().strip()
        bathroom_num = ""
        bedroom_num = ""
        furnished = ""  #mapping to 1 for yes - for no
        if "studio apartment to rent" in property_desc:
            bathroom_num = 1
            bedroom_num = 1
            furnished  = 1
        else:
            digits = []
            prop_list = property_desc.split(" ")
            for find in prop_list:
                if find.isdigit():
                    digits.append(find)
            for item in prop_list:
                item = item.strip()
                if "bedroom" in item:
                    bedroom_num = digits[0]
                if "bathroom" in item:
                    bathroom_num = digits[-1]
            furn_status = property_desc.split(",")[-1].strip()
            if "furnished" == furn_status:
                furnished  = 1
            elif "furnished or unfurnished" == furn_status:
                furnished = 1
            if "unfurnished" == furn_status:
                furnished = 0

        #adding the revelant numbers to their respective lists
        list_bathroom.append(bathroom_num)
        list_bedroom.append(bedroom_num)
        list_furnished.append(furnished)
    
        
            
            


#creating a pandas dataframe
import pandas as pd
d = {"Listed property price(€)": list_price, 'Address':list_address,"Bedroom(s)":list_bedroom,"Bathroom(s)": list_bathroom, "Property desc": list_property_desc,"Week YN" : list_week,"Month YN": list_month, "Furnished Status":list_furnished }
df = pd.DataFrame(data=d)
    

df.head()

,Listed property price(€),Address,Bedroom(s),Bathroom(s),Property desc,Week YN,Month YN,Furnished Status
0,1600,"Earlsfield Court, FRANCIS STREET, Dublin 8",2,1,Available 1st June - Two double bedroom apartm...,0,1,1
1,1500,"Clearwater Court North Royal Canal Park, Royal...",2,1,"Available Now - Super spacious, interior desig...",0,1,1
2,2200,"Gallery Quay, Hanover Quay, Dublin 2",2,2,Available Now - Beautiful dual aspect top floo...,0,1,1
3,1300,"23 Trimleston, Hamelet Lane, Balbriggan, Co. D...",2,1,Available 10th June - Available to move into 1...,0,1,1
4,1650,"Apt.2 Shelbourne Village, Ringsend, Dublin 4.,...",1,1,Available Now - Modern apartment in the heart ...,0,1,1


The column furnished status indicates whether a property is furnished or not.  1 indicates furnished 0 indicates unfurnished.
Adress2 represents a shorter address. this is used if the google places API could not find the longitude and latitude of the orIginal address. I noticed there were some  discrepancies  on the website such as repetition and incorrectly formatted. The following code attempts to correct this.

In [46]:
df["Furnished Status"].value_counts()

# have to convert categorical variablefurnished status tp yes maybe and no and create a bs4 rating


df.info()# has 2723 properties listed

df['Bedroom(s)'] = df['Bedroom(s)'].astype(int)

#removign instances where the website did not contain bathroom number or furnished status  
df = df[df['Bathroom(s)'] != ""]
df = df[df["Furnished Status"] != ""]

df['Bathroom(s)'] = df['Bathroom(s)'].astype(int)

df["Furnished Status"] = df["Furnished Status"].astype(int)
df["Address2"] = ""

df["Longitude"]  = ""
df["latitude"] = ""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5684 entries, 0 to 5683
Data columns (total 8 columns):
Listed property price(€)    5684 non-null int64
Address                     5684 non-null object
Bedroom(s)                  5684 non-null object
Bathroom(s)                 5684 non-null object
Property desc               5684 non-null object
Week YN                     5684 non-null int64
Month YN                    5684 non-null int64
Furnished Status            5684 non-null object
dtypes: int64(3), object(5)
memory usage: 355.3+ KB


In [63]:
i = 0
for item in df["Address"]:
    item = item.split(",")
    if len(item) == 2:
        item = "".join(item)
    elif len(item) == 3:
        item = "".join(item[1:])
    elif len(item) == 4:
        item = "".join(item[2:])
    else:
        item = "".join(item[3:])
    df.iloc[i,8] = item
    i+=1

df_sample was used to understand how to use the google places API. df2 is a sample of 2592 observations from the dataset. I had to do this as google places API has quotes where use is free. Although, beyond this, charges will incur.

In [64]:
df2 = df.sample(2592)
df_sample = df.sample(10)

In [65]:
df2.reset_index(drop=True,inplace=True)
df_sample.reset_index(drop=True,inplace=True)

In [71]:

api_key = os.environ["APIKEY"]
gm = googlemaps.Client(key=api_key)


In [74]:
list_adr = df2["Address"]
list_adr2 = df2["Address2"]
for index,value in list_adr.items():
    index= int(index)
    try:
        gecode_result = gm.geocode(value)[0]
    except IndexError:
        try:
            gecode_result = gm.geocode(list_adr2[index])[0]
        except IndexError:
            try:
                gecode_result = gm.geocode("".join(value.split(",")[-2:]))[0]
            except IndexError:
                gecode_result = gm.geocode("".join(value.split(",")[-1:]))[0]
    if len(gecode_result) >=  1 :
        df2.iloc[int(index),int(9)] = gecode_result["geometry"]['location']['lng']
        df2.iloc[int(index),int(10)] =  gecode_result["geometry"]['location']['lat']
    else:
        print(index,value)
    

In [76]:
cleaned_df = df2.drop(['Address2'],axis=1)

## Adding Extra Functionailty To Our Dataset

Does the location of a property matter that much when renting a house? Many people in Dublin would agree, as driving cars is impractical due to the huge volumes of traffic coming into the city, public transport is not very affordable and biking under windy rain just sucks! Here, I pick three locations O'Connel bridge, IFSC and 'silicon docks'. After this, I found the distance between the location of each property and the revelant point.


In [79]:
geo_df = deepcopy(cleaned_df)

#o connel bridge cordinates
geo_df["lat_bridge"] = 53.347252
geo_df["long_bridge"]= -6.259245

#ifsc
geo_df["lat_ifsc"] = 53.34947
geo_df["long_ifsc"] =  -6.245472

#silicon docks 
geo_df['lat_silicon_docks'] =  53.343497
geo_df['long_silicon_docks']= -6.23762

geo_df['coord'] = geo_df['latitude'].astype(str) + ',' + geo_df['Longitude'].astype(str)
geo_df['coord_bridge'] = geo_df['lat_bridge'].astype(str) + ',' + geo_df["long_bridge"].astype(str)

geo_df['coord_ifsc'] = geo_df['lat_ifsc'].astype(str) + ',' + geo_df["long_ifsc"].astype(str)

geo_df['coord_silicon_docks'] = geo_df['lat_silicon_docks'].astype(str) + ',' + geo_df["long_silicon_docks"].astype(str)

def get_distance(coord1,coord2):
    dist = geopy.distance.distance(coord1, coord2).km
    return dist

geo_df['dist_bridge'] = ""
#[get_distance(**geo_df[['coord','coord_bridge']].iloc[i].to_dict()) for i in range(geo_df.shape[0])]
len(geo_df.columns)

21

In [83]:
for i in range(geo_df.shape[0]):
    geo_df.iloc[i,20 ] = get_distance(geo_df.iloc[i,16],geo_df.iloc[i,17])
    

geo_df['dist_ifsc'] = ""
geo_df['dist_silicon_docks'] = ""

for i in range(geo_df.shape[0]):
    geo_df.iloc[i,21 ] = get_distance(geo_df.iloc[i,16],geo_df.iloc[i,18])
    geo_df.iloc[i,22 ] = get_distance(geo_df.iloc[i,16],geo_df.iloc[i,19])

In [84]:
geo_df

,Listed property price(€),Address,Bedroom(s),Bathroom(s),Property desc,Week YN,Month YN,Furnished Status,Longitude,latitude,...,long_ifsc,lat_silicon_docks,long_silicon_docks,coord,coord_bridge,coord_ifsc,coord_silicon_docks,dist_bridge,dist_ifsc,dist_silicon_docks
0,1600,"Earlsfield Court, FRANCIS STREET, Dublin 8",2,1,Available 1st June - Two double bedroom apartm...,0,1,1,-6.27406,53.3398,...,-6.245472,53.343497,-6.23762,"53.3397711,-6.2740561","53.347252,-6.259245","53.34947,-6.245472","53.343497,-6.23762",1.29085,2.18846,2.46204
1,1500,"Clearwater Court North Royal Canal Park, Royal...",2,1,"Available Now - Super spacious, interior desig...",0,1,1,-6.31753,53.3776,...,-6.245472,53.343497,-6.23762,"53.3775534,-6.3175322","53.347252,-6.259245","53.34947,-6.245472","53.343497,-6.23762",5.14102,5.72553,6.53233
2,2200,"Gallery Quay, Hanover Quay, Dublin 2",2,2,Available Now - Beautiful dual aspect top floo...,0,1,1,-6.23902,53.3429,...,-6.245472,53.343497,-6.23762,"53.3429414,-6.2390153","53.347252,-6.259245","53.34947,-6.245472","53.343497,-6.23762",1.43017,0.844296,0.111624
3,1300,"23 Trimleston, Hamelet Lane, Balbriggan, Co. D...",2,1,Available 10th June - Available to move into 1...,0,1,1,-6.20668,53.6147,...,-6.245472,53.343497,-6.23762,"53.6147491,-6.206679","53.347252,-6.259245","53.34947,-6.245472","53.343497,-6.23762",29.975,29.6364,30.2589
4,1650,"Apt.2 Shelbourne Village, Ringsend, Dublin 4.,...",1,1,Available Now - Modern apartment in the heart ...,0,1,1,-6.23533,53.3419,...,-6.245472,53.343497,-6.23762,"53.3418769,-6.2353345","53.347252,-6.259245","53.34947,-6.245472","53.343497,-6.23762",1.70113,1.08165,0.235971
5,2200,"Kimmage Grove, Kimmage, Dublin 6w",3,1,Available 1st June - Beautiful 3 bed semi deta...,0,1,1,-6.29146,53.3166,...,-6.245472,53.343497,-6.23762,"53.3165996,-6.2914635","53.347252,-6.259245","53.34947,-6.245472","53.343497,-6.23762",4.03047,4.77182,4.6722
6,1508,"Bridge Street, Ringsend, Dublin 4",1,1,Available 2nd July - Mts property are delighte...,0,1,1,-6.22804,53.3416,...,-6.245472,53.343497,-6.23762,"53.3416398,-6.228035999999999","53.347252,-6.259245","53.34947,-6.245472","53.343497,-6.23762",2.17039,1.45186,0.670968
7,2750,"82 Whiteoaks, Clonskeagh, Dublin 14",3,2,Available 1st June - 3 bedroom semi detached h...,0,1,1,-6.23634,53.3027,...,-6.245472,53.343497,-6.23762,"53.30267019999999,-6.236342199999999","53.347252,-6.259245","53.34947,-6.245472","53.343497,-6.23762",5.191,5.24387,4.54451
8,2500,"30 Beverton Green, Donabate, Co. Dublin",4,3,"Available 5th July - This stunning 4 bedroom, ...",0,1,0,-6.15606,53.4913,...,-6.245472,53.343497,-6.23762,"53.4913327,-6.156061999999999","53.347252,-6.259245","53.34947,-6.245472","53.343497,-6.23762",17.4411,16.8704,17.3238
9,1100,"9 Foxborough Walk, Lucan, Co. Dublin",1,1,Available Now - Stunning 1 bedroom furnished a...,0,1,1,-6.42266,53.3406,...,-6.245472,53.343497,-6.23762,"53.3406348,-6.4226569","53.347252,-6.259245","53.34947,-6.245472","53.343497,-6.23762",10.9085,11.8415,12.3286


In [85]:
# dropping columns that were only needed to caclulate the distance
rent_df = deepcopy(geo_df.drop(['lat_bridge','long_bridge','lat_ifsc','long_ifsc','lat_silicon_docks','long_silicon_docks','coord','coord_bridge','coord_ifsc','coord_silicon_docks'],axis=1))

## Area Trendiness: Using The Google Places API

How do we measure the effect of trendiness or popularity in a qauntitative way? Popular areas tend to have better amenities such as resturant's. I choose google places API to get an average review rating of restaurants nearby(1.5km radius) and an average price level.

**Review ratings** have a range of (1 to 5)  based on aggregated user reviews. 

The **price level** of the place, on a scale of 0 to 4. The exact amount indicated by a specific value will vary from region to region. Price levels are interpreted as follows: 
- 0 — Free
- 1 — Inexpensive
- 2 — Moderate
- 3 — Expensive
- 4 — Very Expensive


In [86]:
apikey= os.environ["APIKEY"]
rent_df['coord'] = rent_df['latitude'].astype(str) + ',' + rent_df['Longitude'].astype(str)
my_fields = ['price_level', 'rating']
rent_df["price level"] = ""
rent_df['rating'] = ""

gmaps = googlemaps.Client(apikey')

def search_places(loc,rad,types,open_now=False):
    places = []
    place_results = gmaps.places_nearby(location=loc,radius=rad,type=types,open_now=False)
    places.extend(place_results["results"])
    time.sleep(3)
    while "next_page_token" in place_results:
        place_results = gmaps.places_nearby(location=loc,page_token=place_results['next_page_token'])
        places.extend(place_results['results'])
        time.sleep(3)
    return places

In [ ]:
for index,value in rent_df['coord'].items():
    place_result = search_places(loc =value ,rad=1500,types='resturant')
    list_price_level= []
    list_rating = []
    for place in place_result:
        try:
            rating = place['rating']
        except KeyError:
            rating = None

        try:
            price_level = place['price_level']
        except KeyError:
            price_level = None
        if price_level:
            list_price_level.append(price_level)
        if rating:
            list_rating.append(rating)
    if len(list_price_level) >0:
        rent_df.iloc[index,14] = sum(list_price_level)/len(list_price_level)
    if len(list_rating) > 0:
        rent_df.iloc[index,15] = sum(list_rating)/len(list_rating)


In [91]:
rent_df.to_hdf('collected_rented_data.h5', key='df', mode='w')

C:\Users\tonyr\Anaconda3\lib\site-packages\pandas\core\generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['Address', 'Property desc', 'Longitude', 'latitude', 'dist_bridge', 'dist_ifsc', 'dist_silicon_docks', 'coord', 'price level', 'rating']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
